In [ ]:
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers import Reshape, Dense, Dropout, Flatten, Conv2D, UpSampling2D, ReLU, LeakyReLU, Activation
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import initializers

import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [ ]:
np.random.seed(1000)

noise_dimension = 100
epochs=100
batch_size=128

# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

In [ ]:
# Load MNIST images and labels
# We do not need test data, therefore we use everything for training
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
X_train = np.concatenate([X_train, X_test])

# Normalize data between -1 and 1. 
# This leads to better results together with tanh activation that normalizing between 0 and 1 with sigmoid activation.
X_train = (X_train - 127.5)/127.5
X_train = X_train.reshape((-1, 28, 28, 1))
print('Shape of training data:', X_train.shape)

In [ ]:
# Build generator
generator = Sequential()
generator.add(Dense(128*7*7, input_dim=noise_dimension))
#generator.add(BatchNormalization())
generator.add(Activation('relu'))
generator.add(Reshape((7, 7, 128)))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(64, kernel_size=(5, 5), padding='same'))
#generator.add(BatchNormalization())
generator.add(Activation('relu'))
generator.add(UpSampling2D(size=(2, 2)))
generator.add(Conv2D(1, kernel_size=(5, 5), padding='same'))
#generator.add(BatchNormalization())
generator.add(Activation('tanh'))
generator.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
# Build discriminator
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size=(5, 5), strides=(2, 2), padding='same', input_shape=(28, 28, 1), kernel_initializer=initializers.RandomNormal(stddev=0.02)))
discriminator.add(LeakyReLU(0.2))
#discriminator.add(Dropout(0.3))
discriminator.add(Conv2D(128, kernel_size=(5, 5), strides=(2, 2), padding='same'))
discriminator.add(LeakyReLU(0.2))
#discriminator.add(Dropout(0.3))
discriminator.add(Flatten())
discriminator.add(Dense(1))
discriminator.add(Activation('sigmoid'))
discriminator.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
# Build GAN
discriminator.trainable = False
gan_in = Input(shape=(noise_dimension,))
generator_out = generator(gan_in)
gan_out = discriminator(generator_out)
gan = Model(inputs=gan_in, outputs=gan_out)
gan.compile(loss='binary_crossentropy', optimizer=adam)

In [ ]:
discriminator_losses = []
generator_losses = []

def plot_losses():
    plt.figure(figsize=(10, 10))
    plt.plot(discriminator_losses, label='Discriminator loss')
    plt.plot(generator_losses, label='Generator loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
def plotGeneratedImages():
    noise = np.random.normal(0, 1, size=[25, noise_dimension])
    generated_images = generator.predict(noise)
    plt.figure(figsize=(5, 5))
    dim = (5, 5)
    for i in range(25):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i,:,:,0], cmap='gray_r')
        plt.axis('off')
    plt.show()

In [ ]:
print('Number of epochs to train:', epochs)
print('Training with batch size of:', batch_size)
print('Total training samples:', X_train.shape[0])

num_iterations = int(X_train.shape[0] / batch_size)

for epoch in range(1, epochs+1):
    print('Epoch %d' % epoch)
    
    np.random.shuffle(X_train)
    for i in tqdm(range(num_iterations)):
        
        # Create a training batch containing real images to train discriminator.
        # We train with two separate mini-batches each containg only real or only generated images.
        X_real = X_train[i*batch_size:i*batch_size+batch_size]
        Y_real = np.zeros(batch_size)
        # One-sided label smoothing
        Y_real.fill(0.9)

        discriminator.trainable = True
        discriminator_loss_real = discriminator.train_on_batch(X_real, Y_real)

        # Create a training batch containing generated images to train discriminator
        noise = np.random.normal(0, 1, size=[batch_size, noise_dimension])
        X_generated = generator.predict(noise)
        Y_generated = np.zeros(batch_size)
        discriminator_loss_generated = discriminator.train_on_batch(X_generated, Y_generated)
        
        discriminator_loss = (discriminator_loss_real + discriminator_loss_generated) / 2
        
        # Create a training batch to train the generator
        noise = np.random.normal(0, 1, size=[batch_size, noise_dimension])
        Y_generated = np.zeros(batch_size)
        # We flip the labeling (generated = real) for training the generator.
        # This is because if the discriminator classifies a generated image as real, 
        # the loss of the generator is low.
        Y_generated.fill(0.9)
        discriminator.trainable = False
        generator_loss = gan.train_on_batch(noise, Y_generated)

    # Save loss of last batch from current epoch
    discriminator_losses.append(discriminator_loss)
    generator_losses.append(generator_loss)

In [ ]:
plot_losses()
plotGeneratedImages()